In [18]:
import requests
import json
import re
from bs4 import BeautifulSoup
import fitz
from sklearn.metrics.pairwise import cosine_similarity
import wikipediaapi
user_agent = "QuizBowlPractice/1.0 (ani.aniketdixit_00@yahoo.com)"
wiki_wiki = wikipediaapi.Wikipedia(
    language='en', extract_format= wikipediaapi.ExtractFormat.WIKI, user_agent = user_agent)

In [19]:
def load_pdf_files(search_terms):
    matching = []
    
    url = 'https://collegiate.quizbowlpackets.com/'
    searches = search_terms
    pdf_links = []
    all_pdf_links = []

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        for link in links:
            for search in searches:
                if search in link.text:
                    matching.append(link.get('href'))
    
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    for link in matching: 
        url_link = 'https://collegiate.quizbowlpackets.com/' + link
        link_response = requests.get(url_link)
        if link_response.status_code == 200:
            soup = BeautifulSoup(link_response.text, 'html.parser')
            links = soup.find_all('a')
            pdf_links.append([link['href'] for link in links if link['href'].endswith('.pdf')])
    
    if pdf_links:
        all_pdf_links = [link for sublist in pdf_links for link in sublist]
        return all_pdf_links

In [20]:
def extract_text(pdf_url):
    try:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            pdf_data = response.content
            pdf_document = fitz.open(stream=pdf_data, filetype="pdf")
            pdf_text = ""
            for page in pdf_document:
                pdf_text += page.get_text()
            return pdf_text
        else:
            print(f"Failed to retrieve PDF from {pdf_url}")
            return ""
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return ""






In [21]:
def extract_questions_and_answers(text):
    questions = []
    current_question = None

    # Split the text into lines
    lines = text.strip().split('\n')

    for line in lines:
        line = line.strip()

        # Check if the line starts with a number followed by a period (indicating a question)
        if re.match(r'^\d+\.', line):
            if current_question:
                questions.append(current_question)
            current_question = {"question": line}
        elif line.startswith("ANSWER:"):
            if current_question:
                current_question["answer"] = line[len("ANSWER:"):].strip()
        elif line.startswith("<"):
            # Handle any additional information, e.g., <Marathe, Commonwealth Nonfiction> <Ed.>
            pass

    if current_question:
        questions.append(current_question)

    # Convert the list of questions to JSON
    result = json.dumps(questions, indent=4)
    return result

In [22]:
list_l = ['2023 CLEAN']
loaded = load_pdf_files(list_l)

n = 1
for l in loaded:
    print(f'{n}' + '\n' + '--' * 15)
    text_st = extract_text(l)
    json_st = extract_questions_and_answers(text_st)
    print(json_st)
    print('\n' + '--' * 15)

1
------------------------------
[
    {
        "question": "1. A letter attacking this work as \u201cShame\u2019s Voice\u201d may have been written by its author under the pseudonym Vladimir Dixon.",
        "answer": "Finnegans Wake"
    },
    {
        "question": "2. A character in this story imagines if roses suddenly began to smell like a sweating horse before telling another character \u201cI",
        "answer": "\u201c The Bet \u201d"
    },
    {
        "question": "3. One poem by this author says that \u201cHumanity is a river of light running from the ex-eternity to eternity.\u201d The speaker of a",
        "answer": "Khalil  Gibran"
    },
    {
        "question": "4. Two giant policemen on stilts refuse to guide this character to the nearest police station. In one work, John Bull thinks this",
        "answer": "Berenger"
    },
    {
        "question": "5. One story by this author ends with two characters hearing the \u201crapid discharge of pistol-shots\u201d and k